## Imports

In [1]:
import r5py

from utils import *

In [2]:
centroids = pd.read_parquet('data/centroids.parquet').disp()

143,971 rows x 5 cols; Memory: 24.4 MiB


,geoid,level,city,x,y
,<object>,<object>,<object>,<float32>,<float32>
0,42007,county,pittsburgh,-80.349266,40.682495


In [3]:
r5_modes = {
    'drive':   [r5py.TransportMode.CAR],
    'bike' :   [r5py.TransportMode.BICYCLE],
    'walk' :   [r5py.TransportMode.WALK],
    'transit': [r5py.TransportMode.TRANSIT,
                r5py.TransportMode.WALK]
}

In [4]:
def get_ttm(city, mode,
            departure=dt.datetime(2023, 11, 23, 8, 30),
            levels=('County', 'Tract', 'BG'),
            r5_modes=r5_modes, centroids=centroids):
    t = dt.datetime.now()
    print(f'Getting travel times by {mode} in {city}')
    assert mode in r5_modes, f'Incorrect mode: {mode}'
    city = city.lower().replace(' ', '-').replace('.', '')
    ## build the graph
    network = r5py.TransportNetwork(
        f'data/osm-city/{city}/{city}.osm.pbf',
        (glob(f'data/gtfs/{city}/*.zip')
         if mode == 'Transit' else []))
    times = []
    for level in levels:
        pts = filt(centroids, city=city, level=level.lower())
        pts = pdf2gdf(pts, 'x', 'y', CRS_DEG)
        pts = pts.rename(columns=D(geoid='id'))[['id', 'geometry']]
        ## compute travel times
        computer = r5py.TravelTimeMatrixComputer(
            network, origins=pts, destinations=pts,
            departure=departure,
            transport_modes=r5_modes[mode])
        ttm = computer.compute_travel_times().dropna()
        times.append(ttm.assign(level=level))
    times = (pd.concat(times).reset_index(drop=1)
             .rename(columns=D(from_id='src', to_id='trg',
                               travel_time='time'))
             [['level', 'src', 'trg', 'time']]
             .astype(D(level=CAT, src=CAT, trg=CAT, time=I16)))
    outdir = mkdir(f'data/r5/{city}')
    outpath = outdir / '{}_{}_{}.parquet'.format(
        city, mode, departure.strftime('%Y%m%d-%H%M%S'))
    times.to_parquet(outpath, compression='gzip')
    print(f'Runtime for {city} by {mode}: {dt.datetime.now() - t}')
    return times

In [ ]:
for city in tqdm([
    # 'houston',
    'jacksonville',
    'kansas-city',
    'las-vegas',
]):
    get_ttm(city, 'drive')

  0%|          | 0/3 [00:00<?, ?it/s]

Getting travel times by drive in jacksonville
